In [1]:
import pandas as pd
import sqlite3
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import re
import csv

pd.set_option("display.max_columns", None)

In [2]:
# Data Directory
data_dir = "../../Data/Big-Data-Cup-2021"
nwhl_data_dir = "../../Data/nwhl_pbp"
bucketless_data_dir = '../../Data/bdc/data'
tracking_data_dir = "{}/TrackingData".format(data_dir)
db_path = '/Users/keltim01/Documents/databases/'


### Importing and cleaning the data

Thanks to [The Bucketless](https://twitter.com/the_bucketless) for the great [Tutorial](https://github.com/the-bucketless/bdc/blob/main/bdc_merge_example.ipynb) and the [Cleaned Data](https://github.com/the-bucketless/bdc/tree/main/data) as well as the [hockey_rink](https://github.com/the-bucketless/hockey_rink) package

### Play by Play Data 

In [3]:
# important numbers for the hockey rink 
ICE_LENGTH = 200
ICE_WIDTH = 85
GOAL_X = ICE_LENGTH - 10
GOAL_Y = ICE_WIDTH / 2
D_ZONE = 75
O_ZONE = ICE_LENGTH - 75

In [4]:
# create and connect to database 
conn = sqlite3.connect(db_path + 'bdc_2022.db')
cursor = conn.cursor()

In [5]:
df_pp_info = pd.read_csv("{}/pp_info.csv".format(bucketless_data_dir))


In [6]:
df_pbp = pd.read_csv("{}/pxp_womens_oly_2022_v2.csv".format(bucketless_data_dir))


In [7]:
df_pbp['home_team'] = df_pbp.apply(lambda x: x['team_name'] if x['venue'] == 'home' else x['opp_team_name'], axis=1)
df_pbp['away_team'] = df_pbp.apply(lambda x: x['team_name'] if x['venue'] == 'away' else x['opp_team_name'], axis=1)
df_pbp['game_id'] = df_pbp.loc[:, ['game_date', 'home_team', 'away_team']].sum(axis=1).astype('category').cat.codes
df_pbp['is_shot'] = 0
df_pbp['is_goal'] = 0
df_pbp['event_id'] = df_pbp['event'].astype('category').cat.codes
df_pbp['team_id'] = df_pbp['team_name'].astype('category').cat.codes
df_pbp['player_id'] = df_pbp['player_name'].astype('category').cat.codes
df_pbp['event_successful'] = df_pbp.apply(lambda x: True if x['event_successful']=='t' else False, axis=1)
df_pbp['event_detail_2'] = df_pbp.apply(lambda x: True if x['event_detail_2']=='t' else False, axis=1)
df_pbp['event_detail_3'] = df_pbp.apply(lambda x: True if x['event_detail_3']=='t' else False, axis=1)
df_pbp[f'detail_1_code'] = df_pbp[f'event_detail_1'].astype('category').cat.codes
df_pbp.loc[df_pbp['event']=='Shot', 'is_shot'] = 1
df_pbp.loc[(df_pbp['event']=='Shot') & (df_pbp['event_successful']), 'is_goal'] = 1
df_pbp['goal_diff'] = df_pbp['goals_for'].sub(df_pbp['goals_against'])
df_pbp.loc[df_pbp['period'] == 1,'game_seconds_remaining'] = 2400 + df_pbp.loc[df_pbp['period'] == 1,'clock_seconds'] 
df_pbp.loc[df_pbp['period'] == 2,'game_seconds_remaining'] = 1200 + df_pbp.loc[df_pbp['period'] == 2,'clock_seconds']
df_pbp.loc[df_pbp['period'] == 3,'game_seconds_remaining'] = df_pbp.loc[df_pbp['period'] == 3,'clock_seconds']

df_pbp['event_code'] = df_pbp['event'].astype('category').cat.codes
df_pbp['event_type_code'] = df_pbp['event_type'].astype('category').cat.codes
# mirror home team in even-numbered periods / away team in odd-numbered periods and the opposite for Canada - Usa Game (Thanks to the Bucketless for the code)
df_pbp[['skaters_for','skaters_against']] = df_pbp['situation_type'].str.extract(r'(\d) on (\d)')
df_pbp = df_pbp.astype({'skaters_for': 'int64', 'skaters_against': 'int64'})
df_pbp['strength_state'] = df_pbp['skaters_for'].sub(df_pbp['skaters_against'])

# calculating angles and distances to the goal
diff_x1 = GOAL_X - df_pbp['x_coord']
diff_x1 = diff_x1.astype(float)
diff_y1 = abs(GOAL_Y - df_pbp['y_coord'])
df_pbp['distance_to_goal'] = np.sqrt(diff_x1 ** 2 + diff_y1 ** 2)
df_pbp['angle_to_goal'] = np.divide(diff_x1, diff_y1,out=np.zeros_like(diff_x1),where=(diff_y1 != 0))

# getting the information about the previous event
df_pbp[['prev_event','prev_event_code','prev_event_type','prev_event_type_code','prev_event_x_coord','prev_event_y_coord','prev_event_game_seconds_remaining','prev_event_distance_to_goal','prev_event_angle_to_goal']] = df_pbp[['event','event_code','event_type','event_type_code','x_coord', 'y_coord', 'game_seconds_remaining', 'distance_to_goal', 'angle_to_goal']].shift(1)
df_pbp['time_diff_last_event'] = df_pbp['game_seconds_remaining'] - df_pbp['prev_event_game_seconds_remaining']
df_pbp['angle_diff_last_event'] = df_pbp['angle_to_goal'] - df_pbp['prev_event_angle_to_goal']
df_pbp['distance_diff_last_event'] = df_pbp['distance_to_goal'] - df_pbp['prev_event_distance_to_goal']
# geting the information about the second previous event
df_pbp[['second_prev_event','second_prev_event_code','second_prev_event_type','second_prev_event_type_code','second_prev_event_x_coord','second_prev_event_y_coord','second_prev_event_game_seconds_remaining','second_prev_event_distance_to_goal','second_prev_event_angle_to_goal']] = df_pbp[['event','event_code','event_type','event_type_code','x_coord', 'y_coord', 'game_seconds_remaining', 'distance_to_goal', 'angle_to_goal']].shift(2)
df_pbp['prev_time_diff_last_event'] = df_pbp['prev_event_game_seconds_remaining'] - df_pbp['second_prev_event_game_seconds_remaining']
df_pbp['prev_angle_diff_last_event'] = df_pbp['prev_event_angle_to_goal'] - df_pbp['second_prev_event_angle_to_goal']
df_pbp['prev_distance_diff_last_event'] = df_pbp['prev_event_distance_to_goal'] - df_pbp['second_prev_event_distance_to_goal']

In [8]:
df_pbp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10609 entries, 0 to 10608
Data columns (total 66 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   game_date                                 10609 non-null  object 
 1   season_year                               10609 non-null  int64  
 2   team_name                                 10609 non-null  object 
 3   opp_team_name                             10609 non-null  object 
 4   venue                                     10609 non-null  object 
 5   period                                    10609 non-null  int64  
 6   clock_seconds                             10609 non-null  int64  
 7   situation_type                            10609 non-null  object 
 8   goals_for                                 10609 non-null  int64  
 9   goals_against                             10609 non-null  int64  
 10  player_name                       

In [9]:
df_pbp.to_sql('play_by_play', conn, if_exists='replace')

10609

### Powerplay Info 

In [10]:
df_pp_info.columns

Index(['game_name', 'penalty_number', 'start_video_clock_seconds',
       'end_video_clock_seconds', 'start_period', 'end_period',
       'start_game_clock_seconds', 'end_game_clock_seconds'],
      dtype='object')

In [11]:
df_pp_info.head()

,game_name,penalty_number,start_video_clock_seconds,end_video_clock_seconds,start_period,end_period,start_game_clock_seconds,end_game_clock_seconds
0,2022-02-08 Canada at USA,1,1278,1314,1,1,386,350
1,2022-02-08 Canada at USA,2,1624,1744,1,1,216,96
2,2022-02-08 Canada at USA,3,409,562,2,2,991,871
3,2022-02-08 Canada at USA,4,898,898,2,2,613,613
4,2022-02-08 Canada at USA,5,1690,1927,2,2,272,152


In [12]:
df_pp_info.loc[df_pp_info['start_period'] == 1,'start_game_seconds_remaining'] = 2400 + df_pp_info.loc[df_pp_info['start_period'] == 1,'start_game_clock_seconds'] 
df_pp_info.loc[df_pp_info['start_period'] == 2,'start_game_seconds_remaining'] = 1200 + df_pp_info.loc[df_pp_info['start_period'] == 2,'start_game_clock_seconds']
df_pp_info.loc[df_pp_info['start_period'] == 3,'start_game_seconds_remaining'] = df_pp_info.loc[df_pp_info['start_period'] == 3,'end_game_clock_seconds']
df_pp_info.loc[df_pp_info['end_period'] == 1,'end_game_seconds_remaining'] = 2400 + df_pp_info.loc[df_pp_info['end_period'] == 1,'end_game_clock_seconds'] 
df_pp_info.loc[df_pp_info['end_period'] == 2,'end_game_seconds_remaining'] = 1200 + df_pp_info.loc[df_pp_info['end_period'] == 2,'end_game_clock_seconds']
df_pp_info.loc[df_pp_info['end_period'] == 3,'end_game_seconds_remaining'] = df_pp_info.loc[df_pp_info['end_period'] == 3,'end_game_clock_seconds']

In [13]:
df_pp_info.to_sql('powerplay_info', conn, if_exists='replace')

37

### tracking data 

In [14]:
bucketless_games = [x for x in os.listdir('{}'.format(bucketless_data_dir)) if x.startswith('2022')]
bucketless_games

['2022-02-08 ROC at Finland',
 '2022-02-14 USA at Finland',
 '2022-02-14 Switzerland at Canada',
 '2022-02-08 Canada at USA',
 '2022-02-16 Switzerland at Finland',
 '2022-02-12 Switzerland at ROC']

In [15]:
tracking_games = [x for x in os.listdir('{}'.format(tracking_data_dir)) if bool(re.search('\d{4}', x))]
tracking_games

['2022-02-08 ROC at Finland',
 '2022-02-14 Switzerland at Canada',
 '2022-02-08 Canada at USA',
 '2022-02-14 Finland at USA',
 '2022-02-16 Switzerland at Finland',
 '2022-02-12 Switzerland at ROC']

In [16]:
df_pbp[['game_id', 'home_team', 'away_team','game_date']].value_counts()

game_id  home_team                                       away_team                                       game_date
5        Olympic (Women) - United States                 Olympic (Women) - Canada                        8/2/2022     1826
2        Olympic (Women) - Finland                       Olympic (Women) - United States                 14/2/2022    1810
1        Olympic (Women) - Canada                        Olympic (Women) - Switzerland                   14/2/2022    1765
0        Olympic (Women) - Olympic Athletes from Russia  Olympic (Women) - Switzerland                   12/2/2022    1748
4        Olympic (Women) - Finland                       Olympic (Women) - Olympic Athletes from Russia  8/2/2022     1739
3        Olympic (Women) - Finland                       Olympic (Women) - Switzerland                   16/2/2022    1721
dtype: int64

In [17]:
game_ids = {
'2022-02-08 ROC at Finland': 4,
'2022-02-14 Switzerland at Canada': 1,
'2022-02-08 Canada at USA': 5,
'2022-02-16 Switzerland at Finland':3,
'2022-02-12 Switzerland at ROC': 0,
'2022-02-14 USA at Finland': 2
}

In [18]:
for game in bucketless_games:
        files = [x for x in os.listdir('{}/{}'.format(bucketless_data_dir, game)) if not x.endswith('roster.csv')]
        game_date, away_team, _, home_team = game.split(' ')
        df_roster = pd.read_csv('{}/{}/{} roster.csv'.format(bucketless_data_dir, game, game))
        df_roster = df_roster.rename(columns={'jn': 'jersey_number', 'team':'venue'})
        for file in files:
                df_tracking = pd.read_csv('{}/{}/{}'.format(bucketless_data_dir, game, file))
                df_tracking.loc[df_tracking['period'] == 1,'game_seconds'] = 2400 + df_tracking.loc[df_tracking['period'] == 1,'game_seconds'] 
                df_tracking.loc[df_tracking['period'] == 2,'game_seconds'] = 1200 + df_tracking.loc[df_tracking['period'] == 2,'game_seconds']
                df_tracking['venue'] = df_tracking['team_name'].map({home_team: 'home', away_team: 'away'})
                df_tracking = df_tracking.merge(df_roster, on=['venue','jersey_number'],how="left")
                df_tracking = df_tracking.rename(columns={'x_ft': "x_coord", 'y_ft': "y_coord"})
                df_tracking['game_id'] = game_ids[game]
                df_tracking.to_sql('tracking', conn, if_exists='append')


### old Event data as training data

In [19]:
df_pbp_2021_womens = pd.read_csv("{}/hackathon_womens.csv".format(data_dir))
df_pbp_2021_nwhl = pd.read_csv("{}/hackathon_nwhl.csv".format(data_dir))
df_pbp_2021_womens = df_pbp_2021_womens.append(df_pbp_2021_nwhl, ignore_index=True)


<ipython-input-19-c1d8cb5a3f18>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_pbp_2021_womens = df_pbp_2021_womens.append(df_pbp_2021_nwhl, ignore_index=True)


In [20]:
df_pbp_2021_womens.columns = df_pbp_2021_womens.columns.str.lower().str.replace(' ', '_')
df_pbp_2021_womens = df_pbp_2021_womens.rename(columns={'x_coordinate': 'x_coord', 'y_coordinate': 'y_coord'})

In [21]:
df_pbp_2021_womens['game_id'] = df_pbp_2021_womens.loc[:, ['game_date', 'home_team', 'away_team']].sum(axis=1).astype('category').cat.codes
df_pbp_2021_womens['event_id'] = df_pbp_2021_womens['event'].astype('category').cat.codes
df_pbp_2021_womens['team_id'] = df_pbp_2021_womens['team'].astype('category').cat.codes
df_pbp_2021_womens['player_id'] = df_pbp_2021_womens['player'].astype('category').cat.codes
df_pbp_2021_womens['is_shot'] = 0
df_pbp_2021_womens['is_goal'] = 0
for x in range(1,5):
    df_pbp_2021_womens[f'detail_{x}_code'] = df_pbp_2021_womens[f'detail_{x}'].astype('category').cat.codes
df_pbp_2021_womens.loc[df_pbp_2021_womens['home_team'] == df_pbp_2021_womens['team'], 'is_home'] = 1
df_pbp_2021_womens.loc[df_pbp_2021_womens['event']=='Shot', 'is_shot'] = 1
df_pbp_2021_womens.loc[df_pbp_2021_womens['event']=='Goal', 'is_goal'] = 1
df_pbp_2021_womens['goal_diff'] = df_pbp_2021_womens['home_team_goals'].sub(df_pbp_2021_womens['away_team_goals'])
df_pbp_2021_womens['clock'] = pd.to_datetime(df_pbp_2021_womens['clock'], format='%M:%S')
df_pbp_2021_womens['game_seconds_remaining'] = df_pbp_2021_womens['clock'].dt.minute.mul(60).add(df_pbp_2021_womens['clock'].dt.second)

df_pbp_2021_womens['event_code'] = df_pbp_2021_womens['event'].astype('category').cat.codes
df_pbp_2021_womens['detail_1_code'] = df_pbp_2021_womens['detail_1'].astype('category').cat.codes


In [22]:
df_pbp_2021_womens.loc[df_pbp_2021_womens['team']==df_pbp_2021_womens['home_team'],'strength_state'] = df_pbp_2021_womens.loc[df_pbp_2021_womens['team']==df_pbp_2021_womens['home_team'],'home_team_skaters'].sub(df_pbp_2021_womens.loc[df_pbp_2021_womens['team']==df_pbp_2021_womens['home_team'],'away_team_skaters'])
df_pbp_2021_womens.loc[df_pbp_2021_womens['team']==df_pbp_2021_womens['away_team'],'strength_state'] = df_pbp_2021_womens.loc[df_pbp_2021_womens['team']==df_pbp_2021_womens['away_team'],'away_team_skaters'].sub(df_pbp_2021_womens.loc[df_pbp_2021_womens['team']==df_pbp_2021_womens['away_team'],'home_team_skaters'])

In [23]:
# calculating angles and distances to the goal
diff_2021_x1 = GOAL_X - df_pbp_2021_womens['x_coord']
diff_2021_x1 = diff_2021_x1.astype(float)
diff_2021_y1 = abs(GOAL_Y - df_pbp_2021_womens['y_coord'])

df_pbp_2021_womens['distance_to_goal'] = np.sqrt(diff_2021_x1 ** 2 + diff_2021_y1 ** 2)
df_pbp_2021_womens['angle_to_goal'] = np.divide(diff_2021_x1, diff_2021_y1,out=np.zeros_like(diff_2021_x1),where=(diff_2021_y1 != 0))

# getting the information about the previous event
df_pbp_2021_womens[['prev_event','prev_event_code','prev_event_type','prev_event_type_code','prev_event_x_coord','prev_event_y_coord','prev_event_game_seconds_remaining','prev_event_distance_to_goal','prev_event_angle_to_goal']] = df_pbp_2021_womens[['event','event_code','detail_1','detail_1_code','x_coord', 'y_coord', 'game_seconds_remaining', 'distance_to_goal', 'angle_to_goal']].shift(1)
df_pbp_2021_womens['time_diff_last_event'] = df_pbp_2021_womens['game_seconds_remaining'] - df_pbp_2021_womens['prev_event_game_seconds_remaining']
df_pbp_2021_womens['angle_diff_last_event'] = df_pbp_2021_womens['angle_to_goal'] - df_pbp_2021_womens['prev_event_angle_to_goal']
df_pbp_2021_womens['distance_diff_last_event'] = df_pbp_2021_womens['distance_to_goal'] - df_pbp_2021_womens['prev_event_distance_to_goal']

#getting information about second previous event 
df_pbp_2021_womens[['second_prev_event','second_prev_event_code','second_prev_event_type','second_prev_event_type_code','second_prev_event_x_coord','second_prev_event_y_coord','second_prev_event_game_seconds_remaining','second_prev_event_distance_to_goal','second_prev_event_angle_to_goal']] = df_pbp_2021_womens[['event','event_code','detail_1','detail_1_code','x_coord', 'y_coord', 'game_seconds_remaining', 'distance_to_goal', 'angle_to_goal']].shift(2)
df_pbp_2021_womens['prev_time_diff_last_event'] = df_pbp_2021_womens['prev_event_game_seconds_remaining'] - df_pbp_2021_womens['second_prev_event_game_seconds_remaining']
df_pbp_2021_womens['prev_angle_diff_last_event'] = df_pbp_2021_womens['prev_event_angle_to_goal'] - df_pbp_2021_womens['second_prev_event_angle_to_goal']
df_pbp_2021_womens['prev_distance_diff_last_event'] = df_pbp_2021_womens['prev_event_distance_to_goal'] - df_pbp_2021_womens['second_prev_event_distance_to_goal']

In [24]:
df_pbp_2021_womens.to_sql('pbp_training', conn, if_exists='replace')

50884

In [25]:
df_pbp_2021_womens['prev_event_type'].unique()

array([nan, 'Backhand', 'Retained', 'Dumped', 'Indirect', 'Direct',
       'Carried', 'Lost', 'Played', 'Snapshot', 'Fan', 'Cross-checking',
       'Slapshot', 'Wristshot', 'Deflection', 'Forehand', 'Tripping',
       'Wrap Around', 'Roughing', 'Holding', 'Delay of game', 'Slashing',
       'Too many men on the ice', 'Interference', 'Closing hand on puck',
       'Hooking', 'Goalkeeper interference', 'Boarding', 'High-sticking',
       'Charging', 'Illegal Check to the Head', 'Game Misconduct', 'Feet',
       'Elbowing', 'Unsportsmanlike conduct', 'Face-off violation',
       'Holding the stick'], dtype=object)

In [26]:
df_pbp['event_type'].unique(), df_pbp['prev_event_type'].unique()

(array(['Backhand', nan, 'Dumped', 'Indirect', 'Direct', 'Slapshot',
        'Snapshot', 'Carried', 'Fan', 'Played', 'Wristshot', 'Deflection',
        'Forehand', 'Cross-checking', 'Hooking', 'Boarding',
        'Interference', 'Wrap Around', 'Unsportsmanlike conduct',
        'Tripping', 'Roughing', 'Holding', 'Slashing', 'High-sticking',
        'Feet', 'Checking from behind', 'Goalkeeper interference',
        'Too many men on the ice'], dtype=object),
 array([nan, 'Backhand', 'Dumped', 'Indirect', 'Direct', 'Slapshot',
        'Snapshot', 'Carried', 'Fan', 'Played', 'Wristshot', 'Deflection',
        'Forehand', 'Cross-checking', 'Hooking', 'Boarding',
        'Interference', 'Wrap Around', 'Unsportsmanlike conduct',
        'Tripping', 'Roughing', 'Holding', 'Slashing', 'High-sticking',
        'Feet', 'Checking from behind', 'Goalkeeper interference',
        'Too many men on the ice'], dtype=object))

In [27]:
df_pbp['event'].unique(), df_pbp['prev_event'].unique()

(array(['Faceoff Win', 'Puck Recovery', 'Dump In/Out', 'Zone Entry',
        'Play', 'Shot', 'Takeaway', 'Penalty Taken'], dtype=object),
 array([nan, 'Faceoff Win', 'Puck Recovery', 'Dump In/Out', 'Zone Entry',
        'Play', 'Shot', 'Takeaway', 'Penalty Taken'], dtype=object))

In [28]:
df_pbp_2021_womens['event'].unique(), df_pbp_2021_womens['prev_event'].unique()

(array(['Faceoff Win', 'Puck Recovery', 'Dump In/Out', 'Zone Entry',
        'Play', 'Takeaway', 'Incomplete Play', 'Shot', 'Penalty Taken',
        'Goal'], dtype=object),
 array([nan, 'Faceoff Win', 'Puck Recovery', 'Dump In/Out', 'Zone Entry',
        'Play', 'Takeaway', 'Incomplete Play', 'Shot', 'Penalty Taken',
        'Goal'], dtype=object))

In [29]:
df_pbp[(df_pbp['is_shot']==1) & (df_pbp['player_name'] == 'Alina Muller') & (df_pbp['strength_state'] > 0)]

,game_date,season_year,team_name,opp_team_name,venue,period,clock_seconds,situation_type,goals_for,goals_against,player_name,event,event_successful,x_coord,y_coord,event_type,player_name_2,x_coord_2,y_coord_2,event_detail_1,event_detail_2,event_detail_3,frame_id_1,frame_id_2,home_team,away_team,game_id,is_shot,is_goal,event_id,team_id,player_id,detail_1_code,goal_diff,game_seconds_remaining,event_code,event_type_code,skaters_for,skaters_against,strength_state,distance_to_goal,angle_to_goal,prev_event,prev_event_code,prev_event_type,prev_event_type_code,prev_event_x_coord,prev_event_y_coord,prev_event_game_seconds_remaining,prev_event_distance_to_goal,prev_event_angle_to_goal,time_diff_last_event,angle_diff_last_event,distance_diff_last_event,second_prev_event,second_prev_event_code,second_prev_event_type,second_prev_event_type_code,second_prev_event_x_coord,second_prev_event_y_coord,second_prev_event_game_seconds_remaining,second_prev_event_distance_to_goal,second_prev_event_angle_to_goal,prev_time_diff_last_event,prev_angle_diff_last_event,prev_distance_diff_last_event
4996,12/2/2022,2021,Olympic (Women) - Switzerland,Olympic (Women) - Olympic Athletes from Russia,away,3,737,5 on 4,2,1,Alina Muller,Shot,False,141,26,Wristshot,NaN,NaN,NaN,Blocked,True,False,219.0,NaN,Olympic (Women) - Olympic Athletes from Russia,Olympic (Women) - Switzerland,0,1,0,5,3,6,0,1,737.0,5,26,5,4,1,51.703482,2.969697,Play,3.0,Direct,6.0,132.0,42.0,739.0,58.002155,116.000000,-2.0,-113.030303,-6.298674,Play,3.0,Direct,6.0,150.0,5.0,741.0,54.829280,1.066667,-2.0,114.933333,3.172875
5049,12/2/2022,2021,Olympic (Women) - Switzerland,Olympic (Women) - Olympic Athletes from Russia,away,3,642,5 on 4,2,1,Alina Muller,Shot,False,160,27,Slapshot,NaN,NaN,NaN,On Net,False,False,4887.0,NaN,Olympic (Women) - Olympic Athletes from Russia,Olympic (Women) - Switzerland,0,1,0,5,3,6,2,1,642.0,5,19,5,4,1,33.767588,1.935484,Play,3.0,Direct,6.0,162.0,63.0,643.0,34.702305,1.365854,-1.0,0.569630,-0.934717,Play,3.0,Direct,6.0,134.0,38.0,645.0,56.180513,12.444444,-2.0,-11.078591,-21.478207
10165,16/2/2022,2021,Olympic (Women) - Switzerland,Olympic (Women) - Finland,away,3,960,5 on 4,0,2,Alina Muller,Shot,False,149,20,Wristshot,NaN,NaN,NaN,Blocked,True,False,3642.0,NaN,Olympic (Women) - Finland,Olympic (Women) - Switzerland,3,1,0,5,3,6,0,-2,960.0,5,26,5,4,1,46.768045,1.822222,Zone Entry,7.0,Carried,2.0,126.0,16.0,961.0,69.269402,2.415094,-1.0,-0.592872,-22.501357,Play,3.0,Direct,6.0,32.0,81.0,967.0,162.623030,4.103896,-6.0,-1.688802,-93.353629


In [30]:
df_pbp[(df_pbp['is_shot']==1)].shape

(709, 66)

## NWHL Play-by-Play Data

In [31]:
df_pbp_2017_nwhl = pd.read_csv("{}/nwhl_pbp_1617.csv".format(nwhl_data_dir))
df_pbp_2017_nwhl['Season'] = 2017
df_pbp_2018_nwhl = pd.read_csv("{}/nwhl_pbp_1718.csv".format(nwhl_data_dir))
df_pbp_2018_nwhl['Season'] = 2018
df_pbp_2019_nwhl = pd.read_csv("{}/nwhl_pbp_1819_fixed.csv".format(nwhl_data_dir), delimiter=";").drop(columns=['Unnamed: 43'])
df_pbp_nwhl = pd.concat([df_pbp_2017_nwhl, df_pbp_2018_nwhl, df_pbp_2019_nwhl])

In [32]:
len(df_pbp_2017_nwhl.columns), len(df_pbp_2018_nwhl.columns), len(df_pbp_2019_nwhl.columns), len(df_pbp_nwhl.columns)

(43, 43, 43, 43)

In [33]:
df_pbp_nwhl.head()

,game_id,game_date,home_team,away_team,home_score,away_score,play_index,period,min,sec,game_seconds,event_interval,event_type,event_team,event_player_1,event_player_2,event_player_3,event_detail,x_coord,y_coord,home_skaters,away_skaters,event_angle,event_distance,x_coord_2,y_coord_2,x_coord_1,y_coord_1,home_goalie,away_goalie,plus_player_1,plus_player_2,plus_player_3,plus_player_4,plus_player_5,plus_player_6,minus_player_1,minus_player_2,minus_player_3,minus_player_4,minus_player_5,minus_player_6,Season
0,15083513,2016-10-07,BUF,BOS,0.0,0.0,1,1,20.0,0.0,0.0,0,Faceoff,BOS,Brianna Decker,Emily Janiga,NaN,NaN,0.0594,0.8245,5.0,5.0,0.531130,88.944422,0.0594,0.8245,0.0594,0.8245,Brianne McLaughlin,Brittany Ott,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017
1,15083513,2016-10-07,BUF,BOS,0.0,0.0,2,1,19.0,22.0,38.0,38,Turnover,BOS,Gigi Marvin,NaN,NaN,NaN,-70.6662,-21.5220,5.0,5.0,49.573544,28.272331,-70.6662,-21.5220,70.6662,21.5220,Brianne McLaughlin,Brittany Ott,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017
2,15083513,2016-10-07,BUF,BOS,0.0,0.0,3,1,18.0,29.0,91.0,53,Shot,BUF,Devon Skeats,Brittany Ott,NaN,NaN,-66.6270,1.2750,5.0,5.0,3.261663,22.409301,-66.6270,1.2750,66.6270,-1.2750,Brianne McLaughlin,Brittany Ott,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017
3,15083513,2016-10-07,BUF,BOS,0.0,0.0,4,1,18.0,8.0,112.0,21,BlockedShot,BOS,Zoe Hickel,NaN,NaN,NaN,-46.4112,-20.4595,5.0,5.0,25.659455,47.248249,-46.4112,-20.4595,46.4112,20.4595,Brianne McLaughlin,Brittany Ott,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017
4,15083513,2016-10-07,BUF,BOS,0.0,0.0,5,1,18.0,8.0,112.0,0,Faceoff,BUF,Kristina Lavoie,Rachel Llanes,NaN,NaN,-62.1720,-19.7540,5.0,5.0,36.364969,33.316094,-62.1720,-19.7540,62.1720,19.7540,Brianne McLaughlin,Brittany Ott,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017


In [34]:
df_pbp_nwhl = df_pbp_nwhl.rename(columns={'event_type':'event', 'event_player_1':'player_name', 'event_team':'team_name'})

In [35]:
df_pbp_nwhl['event']

0           Faceoff
1          Turnover
2              Shot
3       BlockedShot
4           Faceoff
           ...     
5071        Faceoff
5072        Faceoff
5073           Shot
5074           Shot
5075        Faceoff
Name: event, Length: 14386, dtype: object

In [36]:
event_code = {
    'Faceoff':1, 
    'Turnover':3, 
    'Shot':7, 
    'BlockedShot':7, 
    'Penalty':4, 
    'Goal':2,
    'GoalieChange':11, 
    'Shootout':12, 
    'ShootoutWon':13
}

df_pbp_nwhl['event_code'] = df_pbp_nwhl['event'].map(event_code)

In [37]:
# calculating angles and distances to the goal
diff_nwhl_x1 = GOAL_X - df_pbp_nwhl['x_coord']
diff_nwhl_x1 = diff_nwhl_x1.astype(float)
diff_nwhl_y1 = abs(GOAL_Y - df_pbp_nwhl['y_coord'])

df_pbp_nwhl['distance_to_goal'] = np.sqrt(diff_2021_x1 ** 2 + diff_2021_y1 ** 2)
df_pbp_nwhl['angle_to_goal'] = np.divide(diff_2021_x1, diff_2021_y1,out=np.zeros_like(diff_2021_x1),where=(diff_2021_y1 != 0))
df_pbp_nwhl['game_seconds_remaining'] = 3600 - df_pbp_nwhl['game_seconds']

# getting the information about the previous event
df_pbp_nwhl[['prev_event','prev_event_code','prev_event_x_coord','prev_event_y_coord','prev_event_game_seconds_remaining','prev_event_distance_to_goal','prev_event_angle_to_goal']] = df_pbp_nwhl[['event','event_code','x_coord', 'y_coord', 'game_seconds_remaining', 'distance_to_goal', 'angle_to_goal']].shift(1)
df_pbp_nwhl['time_diff_last_event'] = df_pbp_nwhl['game_seconds_remaining'] - df_pbp_nwhl['prev_event_game_seconds_remaining']
df_pbp_nwhl['angle_diff_last_event'] = df_pbp_nwhl['angle_to_goal'] - df_pbp_nwhl['prev_event_angle_to_goal']
df_pbp_nwhl['distance_diff_last_event'] = df_pbp_nwhl['distance_to_goal'] - df_pbp_nwhl['prev_event_distance_to_goal']

#getting information about second previous event 
df_pbp_nwhl[['second_prev_event','second_prev_event_code','second_prev_event_x_coord','second_prev_event_y_coord','second_prev_event_game_seconds_remaining','second_prev_event_distance_to_goal','second_prev_event_angle_to_goal']] = df_pbp_nwhl[['event','event_code','x_coord', 'y_coord', 'game_seconds_remaining', 'distance_to_goal', 'angle_to_goal']].shift(2)
df_pbp_nwhl['prev_time_diff_last_event'] = df_pbp_nwhl['prev_event_game_seconds_remaining'] - df_pbp_nwhl['second_prev_event_game_seconds_remaining']
df_pbp_nwhl['prev_angle_diff_last_event'] = df_pbp_nwhl['prev_event_angle_to_goal'] - df_pbp_nwhl['second_prev_event_angle_to_goal']
df_pbp_nwhl['prev_distance_diff_last_event'] = df_pbp_nwhl['prev_event_distance_to_goal'] - df_pbp_nwhl['second_prev_event_distance_to_goal']

df_pbp_nwhl['goal_diff'] = df_pbp_nwhl['home_score'].sub(df_pbp_nwhl['away_score'])
df_pbp_nwhl.loc[df_pbp_nwhl['team_name'] == df_pbp_nwhl['home_team'], 'strength_state'] = df_pbp_nwhl.loc[df_pbp_nwhl['team_name'] == df_pbp_nwhl['home_team'], 'home_skaters'].sub(df_pbp_nwhl.loc[df_pbp_nwhl['team_name'] == df_pbp_nwhl['home_team'], 'away_skaters'])
df_pbp_nwhl.loc[df_pbp_nwhl['team_name'] == df_pbp_nwhl['away_team'], 'strength_state'] = df_pbp_nwhl.loc[df_pbp_nwhl['team_name'] == df_pbp_nwhl['away_team'], 'away_skaters'].sub(df_pbp_nwhl.loc[df_pbp_nwhl['team_name'] == df_pbp_nwhl['away_team'], 'home_skaters'])

df_pbp_nwhl.loc[df_pbp_nwhl['event'] == 'Goal', 'is_goal'] = 1


In [38]:
df_pbp_nwhl.loc[~(df_pbp_nwhl['event'] == 'Goal'), 'is_goal'] = 0

In [39]:
df_pbp_nwhl['is_goal'].value_counts(dropna=False)

0.0    13783
1.0      603
Name: is_goal, dtype: int64

In [40]:
df_pbp_nwhl.loc[(df_pbp_nwhl['event'] == 'Goal') | (df_pbp_nwhl['event'] == 'Shot')].head(50)

,game_id,game_date,home_team,away_team,home_score,away_score,play_index,period,min,sec,game_seconds,event_interval,event,team_name,player_name,event_player_2,event_player_3,event_detail,x_coord,y_coord,home_skaters,away_skaters,event_angle,event_distance,x_coord_2,y_coord_2,x_coord_1,y_coord_1,home_goalie,away_goalie,plus_player_1,plus_player_2,plus_player_3,plus_player_4,plus_player_5,plus_player_6,minus_player_1,minus_player_2,minus_player_3,minus_player_4,minus_player_5,minus_player_6,Season,event_code,distance_to_goal,angle_to_goal,game_seconds_remaining,prev_event,prev_event_code,prev_event_x_coord,prev_event_y_coord,prev_event_game_seconds_remaining,prev_event_distance_to_goal,prev_event_angle_to_goal,time_diff_last_event,angle_diff_last_event,distance_diff_last_event,second_prev_event,second_prev_event_code,second_prev_event_x_coord,second_prev_event_y_coord,second_prev_event_game_seconds_remaining,second_prev_event_distance_to_goal,second_prev_event_angle_to_goal,prev_time_diff_last_event,prev_angle_diff_last_event,prev_distance_diff_last_event,goal_diff,strength_state,is_goal
2,15083513,2016-10-07,BUF,BOS,0.0,0.0,3,1,18.0,29.0,91.0,53,Shot,BUF,Devon Skeats,Brittany Ott,NaN,NaN,-66.6270,1.2750,5.0,5.0,3.261663,22.409301,-66.6270,1.2750,66.6270,-1.2750,Brianne McLaughlin,Brittany Ott,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017,7,92.402651,2.181818,3509.0,Turnover,3.0,-70.6662,-21.5220,3562.0,101.986519,4.040816,-53.0,-1.858998,-9.583867,Faceoff,1.0,0.0594,0.8245,3600.0,90.001389,180.000000,-38.0,-175.959184,11.985130,0.0,0.0,0.0
5,15083513,2016-10-07,BUF,BOS,0.0,0.0,6,1,17.0,51.0,129.0,17,Shot,BUF,Jordyn Burns,Brittany Ott,NaN,NaN,57.8358,1.4960,5.0,5.0,2.748305,31.200086,57.8358,1.4960,57.8358,1.4960,Brianne McLaughlin,Brittany Ott,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017,7,46.970736,0.530120,3471.0,Faceoff,1.0,-62.1720,-19.7540,3488.0,46.970736,0.530120,-17.0,0.000000,0.000000,BlockedShot,7.0,-46.4112,-20.4595,3488.0,92.402651,2.181818,0.0,-1.651698,-45.431916,0.0,0.0,0.0
6,15083513,2016-10-07,BUF,BOS,0.0,0.0,7,1,17.0,43.0,137.0,8,Shot,BOS,Emily Field,Brianne McLaughlin,NaN,NaN,63.8946,-0.2720,5.0,5.0,0.620737,25.106873,63.8946,-0.2720,63.8946,-0.2720,Brianne McLaughlin,Brittany Ott,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017,7,187.238484,8.651163,3463.0,Shot,7.0,57.8358,1.4960,3471.0,46.970736,0.530120,-8.0,8.121042,140.267749,Faceoff,1.0,-62.1720,-19.7540,3488.0,46.970736,0.530120,-17.0,0.000000,0.000000,0.0,0.0,0.0
7,15083513,2016-10-07,BUF,BOS,0.0,0.0,8,1,16.0,45.0,195.0,58,Shot,BOS,Gigi Marvin,Brianne McLaughlin,NaN,NaN,55.4202,-21.1735,5.0,5.0,32.233185,39.697860,55.4202,-21.1735,55.4202,-21.1735,Brianne McLaughlin,Brittany Ott,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017,7,187.238484,8.651163,3405.0,Shot,7.0,63.8946,-0.2720,3463.0,187.238484,8.651163,-58.0,0.000000,0.000000,Shot,7.0,57.8358,1.4960,3471.0,46.970736,0.530120,-8.0,8.121042,140.267749,0.0,0.0,0.0
8,15083513,2016-10-07,BUF,BOS,0.0,0.0,9,1,16.0,43.0,197.0,2,Shot,BUF,Corinne Buie,Brittany Ott,NaN,NaN,-64.6074,-17.6290,5.0,5.0,35.856362,30.096189,-64.6074,-17.6290,64.6074,17.6290,Brianne McLaughlin,Brittany Ott,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017,7,175.625311,4.845070,3403.0,Shot,7.0,55.4202,-21.1735,3405.0,187.238484,8.651163,-2.0,-3.806092,-11.613173,Shot,7.0,63.8946,-0.2720,3463.0,187.238484,8.651163,-58.0,0.000000,0.000000,0.0,0.0,0.0
12,15083513,2016-10-07,BUF,BOS,0.0,0.0,13,1,15.0,7.0,293.0,24,Shot,BOS,Brianna Decker,Brianne McLaughlin,NaN,NaN,80.0712,0.0765,5.0,5.0,0.490886,8.929128,80.0712,0.0765,80.0712,0.0765,Brianne McLaughlin,Brittany Ott,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017,7,183.777719,7.137255,3307.0,Turnover,3.0,38.4516,37.5615,3331.0,132.348971,3.111111,-24.0,4.026144,51.428748,Faceoff,1.0,68.3496,19.7200,3350.0,77.435457,1.629630,-19.0,1.481481,54.913514,0.0,0.0,0.0
13,15083513,2016-10-07,BUF,BOS,0.0,0.0,14,1,15.0,7.0,293.0,0,Shot,BOS,Brianna Decker,Brianne McLaugh

In [41]:
df_pbp_nwhl.to_sql('pbp_nwhl',conn, if_exists='replace', index=False)

14386